## FastICA

FastICA estimates ICA components in two steps:

1. prewhitenning
2. Estimation of the unmixing matrix $\mathbf{W}$ by optimising for negative entropy.

## Why non gaussian?

The goal of ICA is to determine the unmixing matrix which returns source signals that as far as possible non-gaussian. The motivation for this is the Central Limit Theorem, which states:

<center> "_The distribution of a sum of independent random variables tends toward a gaussian distribution_" <center>

<br>
In other words, a mixture of two independent random variables will have a distribution that is closer to Gaussian than that of either of the two original signals. 

Thus, we can use this to form the basis of an algorithm to separate the components. What we are looking for is for each source to be represented as a linear sum of the mixture components with coefficients given by the inverse of our mixing matrix $\mathbf{S}=\mathbf{A^{-1}X}= \mathbf{WX}$

We can try different linear combinations of mixtures $\mathbf{x}$ (where $\mathbf{x}$ is one row of $\mathbf{X}$) e.g.

$$\mathbf{y}= \mathbf{w^Tx}= \sum_i w_i x_i $$

knowing that when we get the right values for $\mathbf{w}$ then it will form a row of the un-mixing matrix $\mathbf{W}$. By re-writing $\mathbf{y}$ as:

$$\mathbf{y}= \mathbf{w^Tx}= \mathbf{w^TAs}= \mathbf{z^Ts} $$

We can see that, in fact, $\mathbf{y}$ can be shown to be a linear combination of the source signals, $\mathbf{s}$. $\mathbf{z^Ts}$ is least Gaussian when only one of the elements of $\mathbf{z}$ is non-zero i.e. when it is equal to one of the source signals. Therefore, by seeking to maximise non-Gaussianity of $\mathbf{w^Tx}$ we can recover $\mathbf{s}$.
    


## So why optimise for negative entropy?

In FastICA maximal non-Gaussianty is, in fact, achieved by optimising to minimis the negatave (neg) entropy, since:

<center> "_a gaussian variable has the largest entropy among all random variables of equal variance (Cover and Thomas, 1991)_" <center>
    
and, estimates of neg-entropy can be shown to be less sensitive to outliers than standard measures of non-gaussianity such as kurtosis. 
    
Negative entropy provides a measure of non-gaussianity which is zero for a gaussian variable and always nonnegative. It is defined as:

$$J(y)=H(y_{gauss}) - H(y) $$

where $y_{gauss}$ is a Gaussian random variable of the same covariance matrix as $y$. Entropy $H$ is defined as:

$$ H(Y)=\sum_i P(Y=a_i) \log P(Y=a_i) $$

## FastICA optimisation

Unfortunately, neg-entropy is, in general, very difficult to computationally estimate because it requires an estimate (possibly nonparametric) of the pdf. Thus, for FastICA [Hyv2000], Hyvarinen instead proposed the following approximate form (based on estimates of maximum entropy) where $v$ is a Gaussian variable of zero mean and unit variance, $y$ is assumed to be of zero mean and unit variance, $E$ is the expectation:

$$J(y) \propto [E[G(y)] -E[G(v)]]^2 $$

Here, $G$ may be virtually any nonquadratic function, but the most robust estimators are found by choosing a $G$ that does not grow too fast e.g. 

$$G(u) = -e^{-u^2/2} $$
    
This leads to the following form of the FastICA algorithm

<img src="imgs/FastICA_1comp.png" style="max-width:100%; width: 40%; max-width: none">

Here, $g(\mathbf{w^Tx})$ is the first derivative of $G(\mathbf{w^Tx})$ and $g'(\mathbf{w^Tx})$ is the second derivative


### Deriving the update equation (non-examinable)

The specific update equation is derived by observing that the function for neg-entropy is maximized for certain optima of the expectation of $G$ ($E[G(\mathbf{w^Tx})]$), which under the constraints that the variance of approximation ($\mathbf{w^Tx}$) must equal unity (since the $\mathbf{x}$ have been whitened):

$$E[(\mathbf{w^Tx}^2)]= ||\mathbf{w}||^2=1 $$

Can be solved for using following equation:

$$F(\mathbf {w})= E[\mathbf{x}g(\mathbf{w^Tx})] - \beta \mathbf {w} = 0 $$

Strictly this is known as a form of ‘kuhn–Tucker' (KT) condition equation, where $\beta$ here is a KT multiplier (similar to a lagrange multiplier) on the gradient of the constraint equation, which allows optimisation derivative of the expection function, under the regularity condition that the magnitude of the variance be 1. 

We can then solve using Newtons method:

$$\mathbf {w^+}= \mathbf {w} -F(\mathbf {w})/JF(\mathbf {w}) $$   

This requires calculation of the Jacobian ($JF(\mathbf {w})= E[\mathbf{xx^T}g'(\mathbf{w^Tx})]$),  which can be simplified using the fact that the data is whitened (and thus $E[\mathbf{xx^T}]=1$ :

$$ E[\mathbf{xx^T}g'(\mathbf{w^Tx})] \approx E[\mathbf{xx^T}]E[g'(\mathbf{w^Tx})]=E[g'(\mathbf{w^Tx})] $$

Leading to the following update:

$$\mathbf {w^+}= \mathbf {w} -\frac{E[\mathbf{x}g(\mathbf{w^Tx})] - \beta \mathbf {w}}{E[g'(\mathbf{w^Tx})]-\beta} $$   

Which through multiply through by $[\beta -E[g'(\mathbf{w^Tx})]]$ reduces to:

$$\mathbf {w^+}=E[\mathbf{x}g(\mathbf{w^Tx})]-E[g'(\mathbf{w^Tx})]\mathbf {w}$$

### Moving to more components

Finally, for estimation of multiple components (independent, and thus decorrelated from one another), the following update protocol is followed. For the $(p+1)$th component:

$$\mathbf{w}_{p+1}= \mathbf{w}_{p+1} =\sum_{j=1}^p \mathbf{w}_{p+1}^T \mathbf{w}_i \mathbf{w}_i \mathbf{w}_j $$ 
$$\mathbf{w}_{p+1}= \frac{\mathbf{w}_{p+1}}{\sqrt{\mathbf{w}_{p+1}^T \mathbf{w}_{p+1}}} $$ 


This decorrrelates outputs after every iteration by subtracting from $\mathbf{w}_{p+1}$ the “projections” of the previously estimated $p$ vectors; followed by normalisation of $\mathbf{w}_{p+1}$